# Train a LSTM to classify yelp reviews

In [1]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
import gensim.downloader as api
from tqdm import tqdm

## Data processing

### Import data

In [2]:
# Read data
df = pd.read_csv('data/raw_reviews/reviews_v1.csv')

In [3]:
# Separate reviews and labels
reviews = df.text
food_labels = df.food
service_labels = df.service

### Get word2vec embeddings

In [4]:
# Tokenize reviews
## seperating the words
from src.data_processing.word_tokenizer import basic_tokenizer
review_list = [basic_tokenizer(review) for review in reviews]

In [5]:
# Embedding reviews
## give each token 300 dim vector from word2vec
from src.data_processing.word_tokenizer import batch_embedding
# Load word2vec model
model = KeyedVectors.load('word2vec/word2vec-google-news-300.model')

In [6]:
# word embeddings for all reviews
x_all = batch_embedding(review_list, model)

Fetching review embeddings: 100%|██████████| 9998/9998 [00:10<00:00, 938.94it/s] 


### Get review labels
Get joint distribution for only food, only service, both food and service, and neither.